In [1]:
import pandas as pd
import numpy as np
import funciones

#base de datos con las denuncias

df_maltrato_animal = pd.read_csv(r"C:\Users\samue\OneDrive\Documents\GitHub\captain_planet_sammy\captain-planet\analisis_maltrato_animal\data\clean\categorizado\df_maltrato_animal_actualizado_categorizado", encoding = 'latin')
df_maltrato_animal[["fecha_de_recepcion", "fecha_de_ratificacion", "fecha_de_turno", "fecha_de_admision/radicacion", "fecha_de_conclusion"]] = df_maltrato_animal[["fecha_de_recepcion", "fecha_de_ratificacion", "fecha_de_turno", "fecha_de_admision/radicacion", "fecha_de_conclusion"]].apply(pd.to_datetime, errors = 'coerce')


In [2]:
df_telefonia_indice = pd.read_csv(r'C:\Users\samue\OneDrive\Documents\GitHub\captain_planet_sammy\captain-planet\analisis_maltrato_animal\data\raw\indice desarrollo social cdmx\telefonia\telefonia-indice-de-desarrollo-social-en-la-ciudad.csv', encoding='utf-8')

funciones.estandarizacion(df_telefonia_indice,'nomgeo','dim')

funciones.limpiar_encabezados(df_telefonia_indice)

df_vivienda_indicee = df_telefonia_indice.sort_values(by=['nomgeo','estratos'])

In [3]:
df_vivienda_indice = pd.read_csv(r'C:\Users\samue\OneDrive\Documents\GitHub\captain_planet_sammy\captain-planet\analisis_maltrato_animal\data\raw\indice desarrollo social cdmx\vivienda-indice-de-desarrollo-social-en-la-ciudad.csv', encoding='utf-8')

funciones.estandarizacion(df_vivienda_indice,'nomgeo','dim')

funciones.limpiar_encabezados(df_vivienda_indice)


In [4]:
df_victimas = pd.read_csv(r'C:\Users\samue\OneDrive\Documents\GitHub\captain_planet_sammy\captain-planet\analisis_maltrato_animal\data\clean\df_victimas.csv', encoding='utf-8')

In [5]:
def preparacion_enriquecimiento(df_og, alcaldias_og, id_denuncia_og):
    df_a_enriquecer = df_og.groupby([alcaldias_og]).count().loc[:,[id_denuncia_og]]

    df_a_enriquecer= df_a_enriquecer.rename(columns={id_denuncia_og:'num_denuncias'})
    return df_a_enriquecer


In [6]:
datos_corr = preparacion_enriquecimiento(df_maltrato_animal, 'alcaldia','id_denuncia')

In [7]:
def enriquecimiento(df_a_enriquecer, df_externo, alcaldias_externo, columnas_de_interes,nombre,tipo):

    if(tipo=='conteo'):
        df_externo_gb = df_externo.groupby([alcaldias_externo]).count()
    elif(tipo=='suma'):
        df_externo_gb = df_externo.groupby([alcaldias_externo]).sum()
    else:
        df_externo_gb = df_externo.groupby([alcaldias_externo]).mean()

    index = 0        

    for columna in columnas_de_interes:

        df_a_enriquecer[nombre[index]] = np.nan

        for i in df_a_enriquecer.index:
            
            datos_externos_temp = df_externo_gb.loc[i]
            
            if(not datos_externos_temp.empty):    
                df_a_enriquecer.loc[i, nombre[index]] = datos_externos_temp[columna]
    
        df_a_enriquecer[nombre[index]] = df_a_enriquecer[nombre[index]].astype('float64')
    
        index = index +1


In [8]:

enriquecimiento(datos_corr, df_maltrato_animal,'alcaldia',['maltrato_fisico'],['maltrato_fisico'],'suma')
enriquecimiento(datos_corr, df_maltrato_animal,'alcaldia',['intemperie_o_inmovilidad'],['intemperie_o_inmovilidad'],'suma')
enriquecimiento(datos_corr, df_maltrato_animal,'alcaldia',['subalimentacion'],['subalimentacion'],'suma')

In [9]:
estrato_bajo_tel = df_telefonia_indice[df_telefonia_indice['estratos']=='Pobreza muy alta']
estrato_medio_tel = df_telefonia_indice[df_telefonia_indice['estratos']=='Satisfaccion mínima']
estrato_alto_tel = df_telefonia_indice[df_telefonia_indice['estratos']=='Estrato alto']

estrato_bajo_viv = df_vivienda_indice[df_vivienda_indice['estratos']=='Pobreza muy alta']
estrato_medio_viv = df_vivienda_indice[df_vivienda_indice['estratos']=='Satisfaccion mínima']
estrato_alto_viv = df_vivienda_indice[df_vivienda_indice['estratos']=='Estrato alto']

In [10]:
enriquecimiento(datos_corr, estrato_bajo_tel, 'nomgeo', ['total'], ['bajo_total_tel'], 'suma')
enriquecimiento(datos_corr, estrato_medio_tel, 'nomgeo', ['total'], ['medio_total_tel'], 'suma')
enriquecimiento(datos_corr, estrato_alto_tel, 'nomgeo', ['total'], ['alto_total_tel'], 'suma')

enriquecimiento(datos_corr, estrato_bajo_viv, 'nomgeo', ['total'], ['bajo_total_viv'], 'suma')
enriquecimiento(datos_corr, estrato_medio_viv, 'nomgeo', ['total'], ['medio_total_viv'], 'suma')
enriquecimiento(datos_corr, estrato_alto_viv, 'nomgeo', ['total'], ['alto_total_viv'], 'suma')


In [11]:
datos_corr['pob_total_tel'] = datos_corr['bajo_total_tel'] + datos_corr['medio_total_tel']+ datos_corr['alto_total_tel']

datos_corr['pctg_alto_tel'] = datos_corr['alto_total_tel'] / datos_corr['pob_total_tel']
datos_corr['pctg_medio_tel'] = datos_corr['medio_total_tel'] / datos_corr['pob_total_tel']
datos_corr['pctg_bajo_tel'] = datos_corr['bajo_total_tel'] / datos_corr['pob_total_tel']

In [12]:
datos_corr['pob_total_viv'] = datos_corr['bajo_total_viv'] + datos_corr['medio_total_viv']+ datos_corr['alto_total_viv']

datos_corr['pctg_alto_viv'] = datos_corr['alto_total_viv'] / datos_corr['pob_total_viv']
datos_corr['pctg_medio_viv'] = datos_corr['medio_total_viv'] / datos_corr['pob_total_viv']
datos_corr['pctg_bajo_viv'] = datos_corr['bajo_total_viv'] / datos_corr['pob_total_viv']

In [14]:
datos_corr.reset_index(inplace=True)
datos_corr = datos_corr.loc[:,['alcaldia','pctg_alto_tel', 'pctg_medio_tel', 'pctg_bajo_tel', 'pctg_alto_viv', 'pctg_medio_viv', 'pctg_bajo_viv']]

In [16]:
datos_corr.to_csv(r'C:\Users\samue\OneDrive\Documents\GitHub\captain_planet_sammy\captain-planet\analisis_maltrato_animal\data\clean\datos_desarrollo')

In [61]:
enriquecimiento(datos_corr, df_victimas,'alcaldiahechos',['idcarpeta'],['num_carpetas'],'conteo')

In [62]:
datos_corr = datos_corr.loc[:,['pctg_alto_tel', 'pctg_medio_tel', 'pctg_bajo_tel', 'pctg_alto_viv', 'pctg_medio_viv', 'pctg_bajo_viv','num_carpetas', 'maltrato_fisico','intemperie_o_inmovilidad','subalimentacion','num_denuncias',]]

In [63]:
datos_corr

,pctg_alto_tel,pctg_medio_tel,pctg_bajo_tel,pctg_alto_viv,pctg_medio_viv,pctg_bajo_viv,num_carpetas,maltrato_fisico,intemperie_o_inmovilidad,subalimentacion,num_denuncias
alcaldia,,,,,,,,,,,
alvaro obregon,0.631740,0.325952,0.042308,0.285429,0.278947,0.435624,27183.0,93.0,587.0,522.0,774
azcapotzalco,0.642261,0.317373,0.040366,0.283898,0.388635,0.327467,16808.0,65.0,371.0,327.0,533
benito juarez,0.796607,0.190274,0.013119,0.582020,0.294547,0.123433,29415.0,72.0,419.0,316.0,600
coyoacan,0.713584,0.257422,0.028995,0.436234,0.276561,0.287205,23512.0,70.0,473.0,408.0,671
cuajimalpa de morelos,0.661551,0.298886,0.039562,0.396824,0.178556,0.424621,5462.0,14.0,136.0,119.0,190
cuauhtemoc,0.650324,0.317112,0.032564,0.361751,0.352035,0.286213,53570.0,88.0,498.0,484.0,743
gustavo a madero,0.594246,0.354137,0.051616,0.258779,0.317495,0.423727,37813.0,132.0,825.0,768.0,1174
iztacalco,0.650549,0.324027,0.025424,0.270853,0.406558,0.322589,15712.0,53.0,359.0,314.0,484
iztapalapa,0.541203,0.395678,0.063119,0.169541,0.302182,0.528277,58687.0,220.0,1315.0,1209.0,1782


In [64]:
import plotly.express as px


fig = px.imshow(datos_corr.corr(), text_auto=True)
fig.show()

-terminar mapas

-checar lo del machine learning (tiempo)
https://www.machinelearningplus.com/time-series/time-series-analysis-python/

-darle a streamlit (diseñar modelo)
    +series tiempo
    +tfi df (https://scikit-learn.org/stable/tutorial/text_analytics/working_with_text_data.html)
    +nubes de palabras
    +correlaciones
    +mapas

-subclases

-aniadir poblacion (peso a las alcaldias)

-word relevance machine learning algorithms

